
<font color="#1874CD"><h1 align="left">**Datamecum**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Árboles de decisión**</h2></font>
<font color="#6E6E6E"><h3 align="left">Notebook: aplicación de las SVM a un problema de clasificación</h3></font>
***

El objetivo de este *notebook* es familiarizarnos con el uso de las SVM en un problema de clasificación. Concretamente, aprenderemos cómo se comportan los diferentes tipos de *kernel* usando problemas fáciles de interpretar y visualizar.

In [1]:
## ---------------------------------- ##
## Librerías 
## ---------------------------------- ##

import numpy as np
import numpy.matlib as matl
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from matplotlib.colors import ListedColormap

import random
random.seed(19)

%matplotlib inline

Ahora, vamos a definir algunas funciones que nos serán de ayuda. La primera, *createDataSet*, se usará para generar problemas en 2D aleatorios con dos clases. Los argumentos de esta función son:

* n: número de puntos.
* model: define qué tipo de separación emplearemos entre las dos clases ('linear', 'square' o 'sine').
* x1_margin: margen de separación entre las dos clases. Cuanto mayor es *x1_margin*, más separadas están las clases. Valores negativos de este parámetro implican superposición entre las dos clases.

In [2]:
def createDataSet(n, model, x1_margin):
    
    X = np.zeros((n,2))
    X[:,0] = np.random.rand(n)*2.0*np.pi

    if model == 'sine':
        X[:,1] = (np.random.rand(n) - 0.5)*2.2
        c = X[:,1] > np.sin(X[:,0])
    elif model == 'linear':
        X[:,1] = np.random.rand(n)*2.0*np.pi
        c = X[:,1] > X[:,0]
    elif model == 'square':
        X[:,1] = np.random.rand(n)*4.0*np.pi*np.pi
        c = X[:,1] > X[:,0]**2
    else:
        X[:,1] = np.random.rand(n)*2.0*np.pi
        c = X[:,1] > X[:,0]
    
    X[c == True,1] += x1_margin
    X[c == False,1] -= x1_margin
    
    return X, c*1

La siguiente función, plotData, se usa para mostrar visualmente el dataset. Los argumentos son:

* X: muestras.
* c: clase de los puntos a mostrar.
* style0: estilo de los puntos de clase 0.
* style1: estilo de los puntos de clase 1.
* title: título de la gráfica.

In [3]:
def plotData(X,c,style0,style1,title):
    
    plt.plot(X[c==0,0],X[c==0,1],style0)
    plt.plot(X[c==1,0],X[c==1,1],style1)
    plt.grid(True)
    plt.axis([X[:,0].min()-0.2, X[:,0].max()+0.2, X[:,1].min()-0.2, X[:,1].max()+0.2])
    plt.xlabel("x0")
    plt.ylabel("x1")
    plt.title(title)

Finalmente, la función *plotModel* muestra la frontera de separación de un clasificador. Sus argumentos son:

* X: muestras.
* clase: clase de los puntos a mostrar.
* clf: clasificador.
* title: título de la gráfica.

In [4]:
def plotModel(X,clase,clf,title):
    
    x0 = X[:,0]
    x1 = X[:,1]
    x0_min, x0_max = x0.min() - .2, x0.max() + .2
    x1_min, x1_max = x1.min() - .2, x1.max() + .2
    hx0 = (x0_max - x0_min)/100.
    hx1 = (x1_max - x1_min)/100.
    xx0, xx1 = np.meshgrid(np.arange(x0_min, x0_max, hx0), np.arange(x1_min, x1_max, hx1))

    if hasattr(clf, "decision_function"):
        z = clf.decision_function(np.c_[xx0.ravel(), xx1.ravel()])
    else:
        z = clf.predict_proba(np.c_[xx0.ravel(), xx1.ravel()])[:, 1]

    z = z.reshape(xx0.shape)
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    
    plt.contourf(xx0, xx1, z, cmap=cm, alpha=.8)
    plt.contour(xx0, xx1, z, [-1, 0, 1], linewidths=[2, 2, 2], colors=['#FF0000', 'k', '#0000FF'])

    plt.scatter(x0[clase==0], x1[clase==0], c='#FF0000')
    plt.scatter(x0[clase==1], x1[clase==1], c='#0000FF')
    plt.gca().set_xlim(xx0.min(), xx0.max())
    plt.gca().set_ylim(xx1.min(), xx1.max())
    plt.grid(True)
    plt.xlabel("x0")
    plt.ylabel("x1")
    plt.title(title)

Una vez definidas las funciones, generamos problemas con dos conjuntos de datos (entrenamiento y test).

In [5]:
## Inputs
n = 500
model = 'linear'
x1_margin = 0.5

In [6]:
## Creamos el dataset
X, clase = createDataSet(n, model, x1_margin)

In [7]:
## Dividimos en entrenamiento y test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, clase, test_size=0.3, random_state=3)

In [ ]:
## Visualizamos

plt.figure(figsize=(12,6))
plt.subplot(121)
plotData(X_train, y_train, 'ro', 'bo', "Training data")
plt.subplot(122)
plotData(X_test, y_test, 'ro', 'bo', "Test data")

Con los datos ya generados, entrenamos un <a href="http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html"> SVC </a>.

El constructor SVC requiere los siguientes argumentos:

* **Tipo de kernel.** Puede ser 'poly', 'linear', 'rbf', ...
* **Parámetro gamma.** Solo para kernels RBF.
* **Parámetro de regularización C.** Cuanto más elevado, tendremos modelos con tasa de error más baja pero más complejos.
* **Grado.** Para kernel polinómico.
* **Coeficinete de orden 0.** Para kernel polinómico.

In [ ]:
clf = SVC(C = 100, kernel='linear')

clf.fit(X_train, y_train)

In [ ]:
## Accuracy (tasa de acierto)

score_train = clf.score(X_train, y_train)
print("Score train =", score_train)
score_test = clf.score(X_test, y_test)
print("Score test =", score_test)

Visualizamos la frontera de clasificación tanto para entrenamiento como test. Destacar que la línea negra es el límite de la frontera, mientras que las líneas azul y roja representan el margen para las clases azul y roja respectivamente.

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plotModel(X_train, y_train, clf, "Training, score = {}".format(score_train))
plt.subplot(1,2,2)
plotModel(X_test, y_test, clf, "Test, score = {}".format(score_test))

Unificando todas las celdas anteriores para hacer pruebas tenemos:

In [ ]:
## Creamos dataset
n = 3000
model = 'sine' ## linear, sine, square
x1_margin = 0.5
X, clase = createDataSet(n, model, x1_margin)

## Partición en entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, clase, test_size=0.3, random_state=3)

## Clasificador
clf = SVC(C=100, kernel='linear')
#clf = SVC(C=10.0, kernel='poly', degree=3, coef0=1.0)
#clf = SVC(C=100.0, kernel='rbf') ## Gamma baja tiende a ser lineal, alto predice bien en acumulación de puntos
clf.fit(X_train, y_train)

## Accuracy
score_train = clf.score(X_train, y_train)
print("Score train =", score_train)
score_test = clf.score(X_test, y_test)
print("Score test =", score_test)

## Visualización de la frontera
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plotModel(X_train, y_train, clf, "Training, score = {}".format(score_train))
plt.subplot(1,2,2)
plotModel(X_test, y_test, clf, "Test, score = {}".format(score_test))

### ¿Qué ocurre si disminuimos el parámetro C?

In [18]:
## Creamos un dataset para los dos ejemplos
n = 1500
model = 'sine'
x1_margin = 0.8
X, clase = createDataSet(n, model, x1_margin)

# Train-test split:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, clase, test_size=0.3, random_state=3)

In [ ]:
## Primer clasificador con C bajo
clf = SVC(C=0.01, kernel='poly', degree=4, coef0=1.0)
clf.fit(X_train, y_train)

## Accuracy
score_train = clf.score(X_train, y_train)
print("Score train =", score_train)
score_test = clf.score(X_test, y_test)
print("Score test =", score_test)

## Visualizamos fronteras
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plotModel(X_train, y_train, clf, "Training, score = {}".format(score_train))
plt.subplot(1,2,2)
plotModel(X_test, y_test, clf, "Test, score = {}".format(score_test))

In [ ]:
## Segundo clasificador
clf = SVC(C=100, kernel='poly', degree=4, coef0=1.0, gamma=0.1)
clf.fit(X_train, y_train)

# Get classification rate:
score_train = clf.score(X_train, y_train)
print("Score train =", score_train)
score_test = clf.score(X_test, y_test)
print("Score test =", score_test)

# Make plots:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plotModel(X_train, y_train, clf, "Training, score = {}".format(score_train))
plt.subplot(1,2,2)
plotModel(X_test, y_test, clf, "Test, score = {}".format(score_test))

### Ejemplo con kernel RBF

In [ ]:
## Creamos un dataset donde los puntos se solapen (x1_margin < 0)
n = 150
model = 'sine'
x1_margin = -0.1
X, clase = createDataSet(n, model, x1_margin)

## Partimos en train - test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, clase, test_size=0.3, random_state=3)

## Clasificador
clf = SVC(C=10.0, kernel='rbf', gamma=0.1) ## "scale" -> busca automática
#clf = SVC(C=1.0, kernel='rbf', gamma=0.1)
#clf = SVC(C=10.0, kernel='rbf', gamma=10)
clf.fit(X_train, y_train)

## Accuracy
score_train = clf.score(X_train, y_train)
print("Score train =", score_train)
score_test = clf.score(X_test, y_test)
print("Score test =", score_test)

## Visualizamos frontera
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plotModel(X_train, y_train, clf, "Training, score = {}".format(score_train))
plt.subplot(1,2,2)
plotModel(X_test, y_test, clf, "Test, score = {}".format(score_test))

## Nota 1: con gamma elevada el modelo solo aprende en acumulaciones de puntos, donde tenemos menor dispersión. Por lo tanto, no generalizamos!
## Nota 2: aunque vemos diferentes fronteras, esto viene de la proyección de la separación lineal en dimensiones más elevadas. Esto es que en el espacio transformado sí tenemos una única frontera lineal
## pero al proyectar en el espacio original podemos obtener diferentes fronteras